# Install Modules

In [10]:
# !pip3 install torch torchvision torchaudio
# !pip install transformers -U
# !pip install sentencepiece
# !pip install py-rouge
# !pip install rouge-metric
# !pip install evaluate
# !pip install bert_score


     ---------------------------------------- 61.1/61.1 kB ? eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Modules

In [1]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from rouge_metric import PyRouge

from evaluate import load

c:\Users\syafi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Data

#### Import Data and Create new Dataframe

In [2]:
data=pd.read_csv('/latest_news_scraper/2.text_preprocessing/combined/combined_file.csv')

new_df = data[['url','clean_text']]
new_df

,url,clean_text
0,https://edition.cnn.com/2023/02/28/sport/los-a...,during the lakers' 27-point comeback victory o...
1,https://edition.cnn.com/2023/02/16/sport/lesle...,it's while running in the moors and hills of h...
2,https://edition.cnn.com/2023/02/27/sport/damia...,damian lillard set an nba record in his monste...
3,https://edition.cnn.com/2023/02/27/football/su...,betrayal has formed part of european politics ...
4,https://edition.cnn.com/2023/02/22/football/pa...,the palestinian team has never taken part in a...
...,...,...
216,https://www.channelnewsasia.com/sport/fa-cup-q...,london : fa cup quarter-final draw made on wed...
217,https://www.channelnewsasia.com/sport/man-unit...,"manchester, england :manchester united fought ..."
218,https://www.channelnewsasia.com/sport/formula-...,statistics for sunday's season-opening bahrain...
219,https://www.channelnewsasia.com/sport/prop-hao...,prop mohamed haouas will miss france's remaini...


# Abstractive Summarization - Pegasus Model

#### Import Pegasus Model using XSum dataset

In [59]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')

#### Testing out on One Article

In [64]:
new_df['clean_text'][164]

'novak djokovic is ranked no 1 in the world for the record 378th time.the 22-time grand slam champion broke the mark set by steffi graf.djokovic, 35, has 6,980 points in the latest atp rankings, 200 more than no 2 carlos alcaraz of spain."it\'s surreal to be that many weeks no 1, to match graf - one of the all-time greats," djokovic said. "just being amongst these legendary names is flattering. i\'m proud of it."djokovic first rose to no 1 at the age of 24 on jul 4, 2011. he spent a personal-best 122 consecutive weeks at the top from 2014-16.djokovic won his men\'s record-tying 22nd major title last month at the australian open. the serbian star returns to action tuesday at the dubai duty free tennis championships.graf, also a 22-time grand slam winner, was no 1 in the women\'s rankings for 377 weeks before retiring in 1999.martina navratilova ranks third on the all-time list with 332 weeks at no 1, with serena williams (319) and roger federer (310) rounding out the top five.'

In [67]:
text="novak djokovic is ranked no 1 in the world for the record 378th time.the 22-time grand slam champion broke the mark set by steffi graf.djokovic, 35, has 6,980 points in the latest atp rankings, 200 more than no 2 carlos alcaraz of spain.'it\'s surreal to be that many weeks no 1, to match graf - one of the all-time greats,' djokovic said. 'just being amongst these legendary names is flattering. i\'m proud of it.'djokovic first rose to no 1 at the age of 24 on jul 4, 2011. he spent a personal-best 122 consecutive weeks at the top from 2014-16.djokovic won his men\'s record-tying 22nd major title last month at the australian open. the serbian star returns to action tuesday at the dubai duty free tennis championships.graf, also a 22-time grand slam winner, was no 1 in the women\'s rankings for 377 weeks before retiring in 1999.martina navratilova ranks third on the all-time list with 332 weeks at no 1, with serena williams (319) and roger federer (310) rounding out the top five."

tokens_input = tokenizer.encode("summarize: "+ text, return_tensors='pt', truncation=True)
ids = model.generate(tokens_input, min_length=50, max_length=200)
summary = tokenizer.decode(ids[0], skip_special_tokens=True)

print(summary)

Serbia's Novak djokovic has broken the all-time record for the most weeks at the top of the men's world rankings, breaking the mark set by one of the sport's all-time greats, steffi graf.


#### Create Function for Summarization with Pegasus model

In [118]:
def summarize(text):
    tokens_input = tokenizer.encode("summarize: "+ text, return_tensors='pt', truncation=True)
    ids = model.generate(tokens_input, min_length=50, max_length=120)
    summary = tokenizer.decode(ids[0], skip_special_tokens=True)
    
    return summary


["One of the NBA's all-time greats has been ruled out for the rest of the season after suffering a foot injury in the Los Angeles lakers' win over the Memphis Grizzlies at the Staples Center in Los Angeles, on Sunday.",
 'As the world marks the 100th anniversary of the end of World War One, the writer and director of one of this year\'s Palme d\'Or contenders talks about the inspiration behind the film\'s opening scene, which she describes as "one of those moments where you know it\'s good and you think: \'oh my god, the uniform\'s more important than the man.\'"',
 "It's not every day you see a player score more than 70 points in a game, but that's exactly what happened in the NBA's all-time classic between the Los Angeles Lakers and Portland Trail blazers at the Staples Center.",
 'The friendship between aleksander eferin and andrea agnelli, then juusvent europe president and club association chairman, was like a "betrayal" between friends, according to the director of a new televisi

#### Perform Summarization on the Entire Dataframe

In [ ]:

text_summary=new_df['clean_text']

text_summary=[summarize(doc) for doc in text_summary]

text_summary

#### Add results to Dataframe

In [119]:
new_df['summary_pegasus']=text_summary2
new_df

C:\Users\syafi\AppData\Local\Temp/ipykernel_27104/1414468045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['summary_pegasus']=text_summary2


,url,clean_text,summary_pegasus
0,https://edition.cnn.com/2023/02/28/sport/los-a...,during the lakers' 27-point comeback victory o...,One of the NBA's all-time greats has been rule...
1,https://edition.cnn.com/2023/02/16/sport/lesle...,it's while running in the moors and hills of h...,As the world marks the 100th anniversary of th...
2,https://edition.cnn.com/2023/02/27/sport/damia...,damian lillard set an nba record in his monste...,It's not every day you see a player score more...
3,https://edition.cnn.com/2023/02/27/football/su...,betrayal has formed part of european politics ...,The friendship between aleksander eferin and a...
4,https://edition.cnn.com/2023/02/22/football/pa...,the palestinian team has never taken part in a...,palestinians are celebrating after their team ...
...,...,...,...
216,https://www.channelnewsasia.com/sport/fa-cup-q...,london : fa cup quarter-final draw made on wed...,Previews and team news for the FA Cup quarter-...
217,https://www.channelnewsasia.com/sport/man-unit...,"manchester, england :manchester united fought ...","manchester united, sheffield united andgrimsby..."
218,https://www.channelnewsasia.com/sport/formula-...,statistics for sunday's season-opening bahrain...,Formula 1 heads to Bahrain for the season-open...
219,https://www.channelnewsasia.com/sport/prop-hao...,prop mohamed haouas will miss france's remaini...,France prop mohamed haouas will miss the rest ...


#### Export to Csv

In [30]:
new_df.to_csv('pegasus_summaryv3.csv', index=False)

# Model Evaluation

### Evaluation of Pegasus using Rouge Metrics

In [54]:
combined_data=pd.read_csv('../abstractive/combined_pegasus_goldtruth.csv')

combined_data.dropna(inplace=True)

In [16]:
len(combined_data)

96

In [49]:
from rouge_metric import PyRouge

# rouge = PyRouge(rouge_n=(1, 2,3, 4), rouge_l=True, rouge_w=True,
#                 rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

rouge = PyRouge(rouge_n=(1, 2,3, 4), rouge_l=True, skip_gap=1)
scores = rouge.evaluate(combined_data['pegasus summary'], combined_data['gold truth'])

In [50]:
scores

{'rouge-1': {'r': 0.06543429193782492,
  'p': 0.0008063830048924492,
  'f': 0.0015931329504556661},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.06543429193782492,
  'p': 0.0008063830048924492,
  'f': 0.0015931329504556661}}

### Evaluation of Pegasus using BERT Score

In [28]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=combined_data['pegasus summary'], references=combined_data['gold truth'], lang="en")
print(results)

{'precision': [0.8446191549301147, 0.8394047617912292, 0.8190465569496155, 0.8665133714675903, 0.8495010137557983, 0.8388718366622925, 0.8636717796325684, 0.845450222492218, 0.8815006017684937, 0.8604516386985779, 0.8297386169433594, 0.8220810890197754, 0.861603319644928, 0.8829523324966431, 0.8431980013847351, 0.8455687165260315, 0.8390985727310181, 0.8382314443588257, 0.8369397521018982, 0.8661029934883118, 0.9009738564491272, 0.8686971068382263, 0.8452743291854858, 0.849420428276062, 0.7448346614837646, 0.8035792112350464, 0.8902730345726013, 0.8607081770896912, 0.8647435307502747, 0.8460294008255005, 0.8679599165916443, 0.8629059791564941, 0.706386387348175, 0.8647406101226807, 0.8642070293426514, 0.8508288264274597, 0.8823255896568298, 0.851731538772583, 0.8586738109588623, 0.8008962869644165, 0.8486719131469727, 0.8834292888641357, 0.8361267447471619, 0.8560142517089844, 0.8622965812683105, 0.8772598505020142, 0.8383579254150391, 0.8342308402061462, 0.7851136326789856, 0.68159389

As the results are given for each row of the dataframe, the codes below is the average of all rows.

#### Precision

In [36]:
total_precision=sum(results['precision'])
p_length=len(results['precision'])
precision=total_precision/p_length
print(round(precision,4))

0.8406


#### Recall

In [38]:
total_recall=sum(results['recall'])
r_length=len(results['recall'])
recall =total_recall/r_length
print(round(recall,4))

0.8457


#### F1 Score

In [39]:
total_f1=sum(results['f1'])
f1_length=len(results['f1'])
f1 =total_f1/f1_length
print(round(f1,4))

0.8429
